<h1>Apache</h1>

<h3>Instalacion de apache</h3>

Utilizar usuario con permisos root, sino crearlo y darle permisos sudoers 

In [ ]:
adduser webuser
gpasswd -a webuser sudo
su - webuser
cd ~
mkdir www

<h3>Instalar dependencias</h3>

Python, Virtualenv

In [ ]:
sudo apt-get install python-pip
sudo pip install virtualenv

sudo apt-get install apache2
sudo apt-get install libapache2-mod-wsgi-py3

<h3>Configuracion de Apache</h3>

In [ ]:
sudo rm /etc/apache2/sites-available/000-default.conf
sudo nano /etc/apache2/sites-available/<<nombre_web>>.conf

In [ ]:
  GNU nano 6.2                    /etc/apache2/sites-available/RoadMaps.conf *                            

<VirtualHost *:80>      
        ServerAdmin patricio
        ServerName 192.168.0.14
        ServerAlias django

        DocumentRoot /home/patricio/www/RoadMaps
        
        ErrorLog ${APACHE_LOG_DIR}/error.log
        CustomLog ${APACHE_LOG_DIR}/access.log combined

        <Directory "/home/patricio/www/RoadMaps">
           Options Indexes FollowSymLinks
           AllowOverride All
           Require all granted
           Allow from All
        </Directory>

        Alias /static /home/patricio/www/RoadMaps/route/static
        <Directory /home/patricio/www/RoadMaps/route/static>
                Require all granted
                Allow from All
        </Directory>

        <Directory /home/patricio/www/RoadMaps/RouteMaps>
                <Files wsgi.py>
                        Require all granted
                        Allow from All
                </Files>
        </Directory>

        WSGIDaemonProcess RoadMaps python-path=/home/patricio/www/RoadMaps/ python-home=/home/patricio/ww>
        WSGIProcessGroup RoadMaps
        WSGIScriptAlias / /home/patricio/www/RoadMaps/RouteMaps/wsgi.py

</VirtualHost>


In [ ]:
sudo a2ensite <<nombre_web>>.conf
sudo service apache2 reload
sudo adduser <<usuario>> www-data
sudo chown -R webuser:www-data /home/<<usuario>>/www

<h3>Comandos apache</h3>

In [ ]:
sudo systemctl start apache2.service
sudo systemctl stop apache2.service
sudo systemctl restart apache2.service
sudo systemctl status apache2.service
sudo apache2ctl configtest

#Logs de ERROR
sudo nano /etc/apache2/sites-available/RoadMaps.conf

# Config Proyecto
sudo nano /etc/apache2/sites-available/<<proyecto>>.conf

<h1>Proyecto</h1>

<h3>Repositorio Git</h3>

Descargar repo de github y crear un entorno vitual dentro de las carpeta principal del proyecto, una vez en el entorno virtual instalar dependencias para el proyecto

In [ ]:
cd ~/www
git clone <<url_del_repo>>
cd <<folder_repo>> 
virtualenv -p python3 venv
source venv/bin/activate

#DENTRO DE EL ENTORNO VIRTUAL

pip install django
pip install djangorestframework
pip install markdown       # Markdown support for the browsable API.
pip install django-filter  # Filtering support
pip install django-qr-code
pip install psycopg



<h3>Configuracion</h3>

Configurar el proyecto una vez instalado y configurado apache, postgres y sus dependencias

Crear archivo para guardar los datos de forma segura en formato .json

In [ ]:
sudo touch /etc/env-production.json
sudo nano /etc/env-production.json

####################################################################

  GNU nano 6.2                             /etc/env-production.json                                       
{
        "ENGINE": "django.db.backends.postgresql",
        "NAME": "road",
        "USER": "postgres",
        "PASSWORD": "route53902",
        "HOST": "localhost",
        "PORT": ""
}

####################################################################

sudo nano <<ruta_settings.py>>/settings.py

####################################################################
  GNU nano 6.2                                    settings.py *                                           

import json

with open('/etc/env-production.json') as env_file:
        ENVIRONMENT = json.load(env_file)

ALLOWED_HOSTS = ['192.168.0.14']

DATABASES = {
    'default':
        {
        'ENGINE': ENVIRONMENT.get('ENGINE'),
        'NAME': ENVIRONMENT.get('NAME'),
        'USER': ENVIRONMENT.get('USER'),
        'PASSWORD': ENVIRONMENT.get('PASSWORD'),
        'HOST': ENVIRONMENT.get('HOST'),
        'PORT': ENVIRONMENT.get('PORT'),
    }
}

STATIC_ROOT = os.path.join(BASE_DIR, 'static_root')
#####################################################################

python manage.py collectstatic

<h1>Firewall</h1>

Chequear firewall, comprobar su estado, dar permisos a apache de no tenerlos 

In [ ]:
sudo ufw COMAND

Usage: ufw COMMAND

Commands:
 enable                          enables the firewall
 disable                         disables the firewall
 default ARG                     set default policy
 logging LEVEL                   set logging to LEVEL
 allow ARGS                      add allow rule
 deny ARGS                       add deny rule
 reject ARGS                     add reject rule
 limit ARGS                      add limit rule
 delete RULE|NUM                 delete RULE
 insert NUM RULE                 insert RULE at NUM
 prepend RULE                    prepend RULE
 route RULE                      add route RULE
 route delete RULE|NUM           delete route RULE
 route insert NUM RULE           insert route RULE at NUM
 reload                          reload firewall
 reset                           reset firewall
 status                          show firewall status
 status numbered                 show firewall status as numbered list of RULES
 status verbose                  show verbose firewall status
 show ARG                        show firewall report
 version                         display version information

Application profile commands:
 app list                        list application profiles
 app info PROFILE                show information on PROFILE
 app update PROFILE              update PROFILE
 app default ARG                 set default application policy


<h1>PostgrestSQL</h1>

<h3>Instalacion</h3>

In [ ]:
sudo apt-get install postgresql postgresql-contrib

<h3>Configuracion</h3>

Desde postgrest se crea la base de datos y se le da permisos al usuario postgrest con contraseña redefinida 

In [ ]:
sudo -u postgres psql

postgres=# CREATE DATABASE road;
postgres=# \connect road;
postgres=# ALTER USER postgres with encrypted password 'route53902';
postgres=# GRANT ALL PRIVILEGES ON road TO postgrest
postgres=# \q
